In [1]:
from crewai.tools import tool

from langchain_tavily import TavilySearch
from langchain_community.tools import DuckDuckGoSearchResults



class SearchTools:
  @tool("Pesquisa na internet")
  def search_tavily(query: str = "") -> str:
      """
      Search the web using Tavily API.
      Recommended for more structured and recent information.
      """
      search_tavily = TavilySearch(max_results=4)
      search_res = search_tavily.invoke(query)
      return search_res

  @tool("Pesquisa na internet com DuckDuckGo")
  def search_duckduckgo(query: str):
      """
      Search the web using DuckDuckGo.
      Returns a list of search results.
      """
      search_tool = DuckDuckGoSearchResults(num_results=4, verbose=True)
      return search_tool.run(query)

In [2]:
class CalculatorTools:
    @tool("Faça um cálculo")
    def calculate(operation):
        """Useful to perform any mathematical calculations,
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        try:
            return eval(operation)
        except SyntaxError:
            return "Erro: Sintaxe inválida"

CalculatorTools.calculate

Tool(name='Faça um cálculo', description='Tool Name: Faça um cálculo\nTool Arguments: {}\nTool Description: Useful to perform any mathematical calculations,\n        like sum, minus, multiplication, division, etc.\n        The input to this tool should be a mathematical\n        expression, a couple examples are `200*7` or `5000/2*10`\n        ', env_vars=[], args_schema=<class 'abc.Façaumcálculo'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x000001E677AF80D0>, result_as_answer=False, max_usage_count=None, current_usage_count=0, func=<function CalculatorTools.calculate at 0x000001E61C8E2200>)

In [3]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import List, Dict, Any
import os
# Optional: environment variables via .env
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass
# =========================
# Config & Helpers
# =========================
# Aceita ambos nomes de variáveis para compatibilidade:
# - OLLAMA_ENDPOINT ou OLLAMA_BASE_URL
# - OLLAMA_MODEL ou LLM_MODEL_NAME
OLLAMA_BASE_URL = os.getenv("OLLAMA_ENDPOINT", os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"))
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", os.getenv("LLM_MODEL_NAME", "phi4"))
LLM_TEMPERATURE = float(os.getenv("LLM_TEMPERATURE", "0.1"))

# Para CrewAI via LiteLLM, o nome do modelo deve conter o provedor (ex.: "ollama/phi4")
PROVIDER_PREFIX = os.getenv("PROVIDER_PREFIX", "ollama/")
CREW_MODEL = f"{PROVIDER_PREFIX}{OLLAMA_MODEL}"  # "ollama/phi4"

# =========================
# LLM p/ Chat direto (LangChain)
# =========================
_llm_instance = None

def load_llm() -> ChatOllama:
    """Instância única para o utilitário 'model_response' (chat direto)."""
    global _llm_instance
    if _llm_instance is None:
        _llm_instance = ChatOllama(
            model=OLLAMA_MODEL,               # "phi4"
            base_url=OLLAMA_BASE_URL,         # "http://localhost:11434"
            temperature=LLM_TEMPERATURE,
        )
    return _llm_instance

def model_response(system_prompt: str, user_query: str,
                   chat_history: List[List[str]] | None = None) -> str:
    llm = load_llm()
    messages = [("system", system_prompt)]
    if chat_history:
        messages += chat_history
    messages.append(("user", user_query))
    prompt_template = ChatPromptTemplate.from_messages(messages)
    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"input": user_query})

c:\Users\niels\anaconda3\envs\GPU_Tensor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from crewai import Agent, Task, Crew, Process, LLM
# =========================
# LLM p/ CrewAI
# =========================
CREW_LLM = LLM(
    model=CREW_MODEL,            # "ollama/phi4"  <- importante o prefixo do provedor
    base_url=OLLAMA_BASE_URL,    # ex.: "http://localhost:11434"
    temperature=LLM_TEMPERATURE,
)

### Especialista em informações da cidade

In [5]:
from crewai import Agent
from textwrap import dedent

city_info_agent = Agent(
    role="Especialista em informações da cidade",
    goal=dedent(
        """Reunir informações úteis sobre o destino escolhido, com base nos interesses e datas da viagem,
        ajudando viajantes a entender o contexto geral do local."""
    ),
    backstory=dedent(
        """
        Sou um especialista com amplo conhecimento sobre cidades ao redor do mundo,
        capaz de fornecer detalhes atualizados sobre clima, eventos culturais, segurança,
        costumes locais e outros aspectos práticos para quem pretende visitar o local.
        Sou apaixonado por compartilhar as melhores experiências e 'joias escondidas' do local.
        """
    ),
    llm=CREW_LLM,
    tools=[SearchTools.search_duckduckgo],
    verbose=True,
    max_iter=10,
    allow_delegation=False,
)

### Logística e informações da cidade

In [6]:
logistics_expert_agent = Agent(
    role="Especialista em logística de viagem",
    goal="Identificar as melhores opções logísticas para a viagem, com foco em praticidade, custo-benefício e conforto.",
    backstory=dedent(
        """
        Sou um profissional focado em planejar a logística da viagem: transporte, hospedagem e deslocamento local.
        Tenho conhecimento sobre companhias aéreas, apps de mobilidade, regiões seguras para se hospedar e otimização de trajetos.
        """
    ),
    llm=CREW_LLM,
    tools=[
        SearchTools.search_duckduckgo,
        CalculatorTools.calculate
    ],
    verbose=True,
    max_iter=10,
    allow_delegation=False,
)

### Especialista em criação de itinerário

In [7]:
itinerary_planner_agent = Agent(
    role="Planejador de itinerário personalizado",
    goal="Criar um roteiro completo com base nas preferências do usuário.",
    backstory=dedent(
        """
        Sou um guia profissional apaixonado por viagens, especialista em organização de itinerários personalizados.
        Minha missão é integrar dados sobre clima, atrações, eventos e logística em uma experiência otimizada para o turista.
        """
    ),
    llm=CREW_LLM,
    tools=[SearchTools.search_duckduckgo],
    verbose=True,
    max_iter=10,
    allow_delegation=False,
)

## Criação das Tarefas (Tasks)

In [16]:
from_city = "Natal" 
destination_city = "Argentina" 
date_from = "23 de novembro de 2025" 
date_to = "30 de novembro de 2025" 
interests = "arte, tecnologia, vinhos, arquitetura" 

print(f"""
Viajando de: {from_city}
Para: {destination_city}
Chegada (início da viagem): {date_from}
Partida (fim da viagem): {date_to}
""")


Viajando de: Natal
Para: Argentina
Chegada (início da viagem): 23 de novembro de 2025
Partida (fim da viagem): 30 de novembro de 2025



### Task: Coleta informações da cidade

In [17]:
from crewai import Task

def city_info_task(agent, from_city, destination_city, interests, date_from, date_to):
    return Task(
        description=dedent(
            f"""
            Levantar informações detalhadas sobre a cidade e coletar dados úteis sobre clima, segurança e costumes locais.
            Identificar marcos culturais, pontos históricos, locais de entretenimento, experiências gastronômicas e quaisquer atividades que se alinhem às preferências do usuário.
            Também destacar eventos e festivais sazonais que podem ser de interesse durante a visita do viajante.
            Use as ferramentas disponíveis para buscar fontes atualizadas e confiáveis.

            Viajando de: {from_city}
            Para: {destination_city}
            Interesses do viajante: {interests}
            Chegada: {date_from}
            Partida: {date_to}

            Seja criterioso, como se estivesse ajudando alguém a ter uma experiência inesquecível.
            """
        ),
        expected_output=dedent(
            f"""
            Um guia detalhado (em formato markdown) em português que inclui:
            - Resumo da cidade e sua cultura;
            - Uma lista selecionada de lugares recomendados para visitar e eventos (se houver);
            - Um detalhamento das despesas diárias, como custos médios com alimentação;
            - Recomendações de segurança;
            - Dicas de costumes locais;
            """
        ),
        agent=agent,
        output_file='relatorio_local.md',
    )

### Task: Planejamento de logística

In [18]:
def plan_logistics_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description=dedent(
            f"""
            Planejar a logística da viagem.
            Identificar as melhores opções de hospedagem, voos e transporte local considerando:

            Destino: {destination_city}
            Data de chegada: {date_from}
            Data de partida: {date_to}
            Interesses: {interests}

            Avalie preço, localização, conveniência e segurança.
            """
        ),
        expected_output=dedent(
            f"""
            Relatório em português (em formato markdown):
            - Sugestão de hospedagens, preferencialmente com localização estratégica;
            - Opções de voos ou meios de transporte para chegada/partida;
            - Sugestões de deslocamento dentro da cidade;
            - Estimativas de custo para cada item;
            """
        ),
        context=context,
        agent=agent,
        output_file='relatorio_logistica.md',
    )

### Task: Construção itinerário final

In [19]:
def build_itinerary_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description=dedent(
            f"""
            Esta tarefa sintetiza todas as informações para criar o roteiro final da viagem.
            Com base nas informações dos outros agentes, desenvolva um itinerário detalhado.
            Cada dia deve conter atividades, clima, transporte, refeições e estimativa de gastos.

            Destino: {destination_city}
            Interesses: {interests}
            Data de chegada: {date_from}
            Data de partida: {date_to}
            """
        ),
        expected_output=dedent("""
        Documento em português que inclui (em formato markdown):
        - Boas vindas e apresentação da cidade em até 2 parágrafos.
        - Programação diária com sugestões de horários e atividades
        - Atrações distribuídas por região e logística
        - Clima previsto, sugestões de transporte e alimentação
        - Eventos e festivais sazonais (se houver)
        - Estimativa de gastos por dia, custo médio das despesas diárias e pontos turísticos.
        - Visão geral dos destaques da cidade com base nas recomendações do guia.
        - Outras dicas e informações adicionais para uma viagem e estadia tranquila.
        """),
        context=context,
        agent=agent,
        output_file='roteiro_viagem.md',
    )

### Executando a criação das tasks

In [20]:
city_info = city_info_task(
  city_info_agent,
  from_city,
  destination_city,
  interests,
  date_from,
  date_to
)

In [21]:
plan_logistics = plan_logistics_task(
  [city_info],
  logistics_expert_agent,
  destination_city,
  interests,
  date_from,
  date_to
)

In [22]:
build_itinerary = build_itinerary_task(
  [city_info, plan_logistics],
  itinerary_planner_agent,
  destination_city,
  interests,
  date_from,
  date_to,
)

## Criação da equipe (Crew)

In [23]:
from crewai import Crew, Process

crew = Crew(
  agents=[city_info_agent, logistics_expert_agent, itinerary_planner_agent],
  tasks=[city_info, plan_logistics, build_itinerary],
  process=Process.sequential,
  full_output=True,
  share_crew=False,
  verbose=True,
  max_rpm=15
)
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5666342b-70b2-4090-abce-2d4de0c0c958                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Especialista em informações da cidade                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Levantar informações detalhadas sobre a cidade e coletar dados úteis sobre clima, segurança e costumes         │
│  locais.                                                                                                        │
│  Identificar marcos culturais, pontos históricos, locais de entretenimento, experiências gastronômicas e        │
│  quaisquer atividades que se alinhem às preferências do usuário.                                                │
│  Também destacar eventos e festivais sazonais que podem ser de interesse durante a visita do viajante.          │
│  Use as ferramentas disponíveis para buscar fontes atualizadas e confiáveis.                                    │
│                                                                                                                 │
│  Viajando de: Natal                                                                                             │
│  Para: Argentina                                                                                                │
│  Interesses do viajante: arte, tecnologia, vinhos, arquitetura                                                  │
│  Chegada: 23 de novembro de 2025                                                                                │
│  Partida: 30 de novembro de 2025                                                                                │
│                                                                                                                 │
│  Seja criterioso, como se estivesse ajudando alguém a ter uma experiência inesquecível.                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\niels\anaconda3\envs\GPU_Tensor\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

c:\Users\niels\anaconda3\envs\GPU_Tensor\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[32;1m[1;3msnippet: La diversidad está dada por la gran cantidad de inmigrantes que llegaron a la región durante 
los siglos XIX y XX, lo que convirtió a Buenos Aires en un crisol cultural . Buenos Aires es una de las principales
metrópolis culturales de Occidente., title: Cultura de la ciudad de Buenos Aires - Wikipedia, la ..., link: 
https://es.wikipedia.org/wiki/Cultura_de_la_Ciudad_de_Buenos_Aires, snippet: 2 de mai. de 2025 — A cultura 
argentina é uma cativante mistura de tradição e modernidade . Das danças apaixonadas do tango às obras de arte 
emocionais de seus museus, o país ..., title: Explore a Argentina: Cultura, gastronomia e o melhor do país, link: 
https://www.cvc.com.br/dicas-de-viagem/inspiracoes/explore-a-argentina-cultura-gastronomia-e-o-melhor-do-pais/, 
snippet: 30 de mar. de 2025 — Sobre Buenos Aires ... O berço do tango e da vanguarda . A das noites intermináveis, 
dos jogos de futebol mais emocionantes e da história. A cidade que nunca dorme ..., title: Turismo na cidade de 
Buenos Aires, link: https://www.argentina.travel/pt/buenos-aires/caba, snippet: 13 de mar. de 2025 — A capital 
argentina encanta com atrações que vão dos tradicionais shows de tango a passeios históricos e culturais, como a 
Casa Rosada, o Caminito, a Plaza de ..., title: Buenos Aires: destino barato e pertinho do Brasil para sua ..., 
link: https://blogvmzviagens.com.br/buenos-aires-destino-barato-perto-brasil-primeira-viagem/[0m
🚀 Crew: crew
└── 📋 Task: 713ea4f5-db09-421b-b6ac-0a8d2ecae86e
    Status: Executing Task...
    └── 🔧 Used Pesquisa na internet com DuckDuckGo (1)

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Planejador de itinerário personalizado                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  To create a comprehensive itinerary for Buenos Aires from November 23 to November 30, 2025, I will integrate   │
│  information about attractions, climate, transportation, dining, and estimated expenses. Here's how the plan    │
│  unfolds:                                                                                                       │
│                                                                                                                 │
│  ### Thought Process                                                                                            │
│                                                                                                                 │
│  1. **Accommodation**: Choose between a hotel in Palermo, an apartment in Puerto Madero, or a hostel in La      │
│  Boca based on budget and proximity to interests.                                                               │
│  2. **Transportation**: Plan for flights to Buenos Aires and local transportation options like public transit,  │
│  car services, and bike rentals.                                                                                │
│  3. **Daily Itinerary**: Develop activities around the user's interests: art, technology, wine, and             │
│  architecture.                                                                                                  │
│  4. **Climate Considerations**: Prepare for November weather with appropriate clothing recommendations.         │
│  5. **Events and Festivals**: Check for any cultural events during this period.                                 │
│  6. **Dining Options**: Suggest local culinary experiences.                                                     │
│  7. **Cost Estimates**: Provide daily expense estimates.                                                        │
│                                                                                                                 │
│  ### Action Plan                                                                                                │
│                                                                                                                 │
│  1. **Accommodation**:                                                                                          │
│     - Hotel Boutique in Palermo: USD 150/night                                                                  │
│     - Apartment in Puerto Madero: USD 200/night                                                                 │
│     - Hostel in La Boca: USD 50/night                                                                           │
│                                                                                                                 │
│  2. **Transportation to Buenos Aires**:                                                                         │
│     - International flights estimated at USD 600-1000.                                                          │
│                                                                                                                 │
│  3. **Local Transportation**:                                                                                   │
│     - Public transport: USD 0.5/day with daily passes.                                                          │
│     - Car services and bike rentals as needed.         

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3ec42d1c-1a11-4456-af51-9ce64fb1a896                                                                     │
│  Agent: Planejador de itinerário personalizado                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5666342b-70b2-4090-abce-2d4de0c0c958                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: To create a comprehensive itinerary for Buenos Aires from November 23 to November 30, 2025, I    │
│  will integrate information about attractions, climate, transportation, dining, and estimated expenses. Here's  │
│  how the plan unfolds:                                                                                          │
│                                                                                                                 │
│  ### Thought Process                                                                                            │
│                                                                                                                 │
│  1. **Accommodation**: Choose between a hotel in Palermo, an apartment in Puerto Madero, or a hostel in La      │
│  Boca based on budget and proximity to interests.                                                               │
│  2. **Transportation**: Plan for flights to Buenos Aires and local transportation options like public transit,  │
│  car services, and bike rentals.                                                                                │
│  3. **Daily Itinerary**: Develop activities around the user's interests: art, technology, wine, and             │
│  architecture.                                                                                                  │
│  4. **Climate Considerations**: Prepare for November weather with appropriate clothing recommendations.         │
│  5. **Events and Festivals**: Check for any cultural events during this period.                                 │
│  6. **Dining Options**: Suggest local culinary experiences.                                                     │
│  7. **Cost Estimates**: Provide daily expense estimates.                                                        │
│                                                                                                                 │
│  ### Action Plan                                                                                                │
│                                                                                                                 │
│  1. **Accommodation**:                                                                                          │
│     - Hotel Boutique in Palermo: USD 150/night                                                                  │
│     - Apartment in Puerto Madero: USD 200/night                                                                 │
│     - Hostel in La Boca: USD 50/night                                                                           │
│                                                                                                                 │
│  2. **Transportation to Buenos Aires**:                                                                         │
│     - International flights estimated at USD 600-1000.                                                          │
│                                                                                                                 │
│  3. **Local Transportation**:                                                                                   │
│     - Public transport: USD 0.5/day with daily passes.

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

## Visualizar resultado

In [24]:
from IPython.display import display, Markdown

def load_markdown(file_path):
  try:
    with open(file_path, 'r', encoding='utf-8') as file:
      content = file.read()
      content = content.replace("```markdown", "").replace("```", "")
      return content
  except Exception as e:
    print(f"Erro ao carregar o arquivo: {str(e)}")
    return None

In [25]:
def display_markdown(content):
  return display(Markdown(content))

In [28]:
md = load_markdown("roteiro_viagem.md")
display_markdown(md)

## Roteiro Buenos Aires: 23 a 30 de novembro de 2025

### Boas-vindas a Buenos Aires

A capital argentina combina charme europeu com vibração latina.
Conhecida como a "berço do Tango", oferece arte, arquitetura, música e
gastronomia únicas.

### Itinerário Dia a Dia

**Dia 1 -- Chegada**\
- Chegada ao Aeroporto Internacional de Ezeiza.\
- Transfer para o hotel/apartamento/hostel.\
- Passeio leve pelo bairro de hospedagem.\
- Jantar em restaurante local.

**Dia 2 -- Arte e Arquitetura**\
- Manhã: Museu Nacional de Belas Artes.\
- Tarde: Caminito em La Boca.\
- Noite: Jantar em Palermo Soho.\
- Transporte: metrô ou caminhada.

**Dia 3 -- Tecnologia e Cultura Moderna**\
- Manhã: Centro Cultural Kirchner.\
- Tarde: Fábrica de Arte (arte contemporânea).\
- Noite: Jantar em Puerto Madero.\
- Transporte: metrô ou bicicleta.

**Dia 4 -- Experiência com Vinhos**\
- Excursão a vinícola próxima para degustação e tour.\
- Almoço na vinícola e jantar ao retornar.

**Dia 5 -- Tango e Música**\
- Manhã: Tempo livre ou compras em Recoleta.\
- Tarde/noite: Show de tango no El Viejo Almacén.\
- Jantar próximo.\
- Transporte: metrô.

**Dia 6 -- Sítios Históricos**\
- Manhã: Plaza de Mayo e Casa Rosada.\
- Tarde: Teatro Colón (visita ou apresentação).\
- Noite: Jantar em San Telmo.

**Dia 7 -- Lazer e Exploração**\
- Manhã: Relaxar no Parque Lezama ou passeio de bicicleta.\
- Tarde: MALBA -- Museu de Arte Latino-Americana.\
- Noite: Jantar de despedida.

**Dia 8 -- Partida**\
- Check-out e traslado para o aeroporto.

### Estimativa de Custos

-   **Hospedagem** (7 noites):
    -   Hotel boutique: \~USD 1.050\
    -   Apartamento: \~USD 1.400\
    -   Hostel: \~USD 350\
-   **Transporte local**: \~USD 3,50/dia\
-   **Alimentação**: \~USD 20/dia\
-   **Atividades**: \~USD 50/dia

### Dicas Adicionais

-   **Clima**: Temperatura amena (10 °C a 20 °C). Leve roupas em
    camadas.\
-   **Eventos**: Consulte a agenda cultural local para festivais ou
    exposições.\
-   **Segurança**: Fique atento em áreas movimentadas e use transporte
    confiável.
